In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
# these three imports are aliased as np, pd, and plt by convention
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# other imports can go here
from datetime import datetime

In [4]:
df = pd.read_csv('revs_scored_jan.tsv', sep='\t', header=0)

In [5]:
print(df)

         rev_timestamp   page_id     rev_id prev_rev_id  is_minor  \
0           1546300800  47649229  876227289   869530001      True   
1           1546300801  47642850  876227292   869530006      True   
2           1546300801  59537875  876227290        None     False   
3           1546300801  60902958  876227291   869530004      True   
4           1546300802  31392285  876227293   876069768      True   
...                ...       ...        ...         ...       ...   
4766347     1548979197   1904845  881187547   868089901      True   
4766348     1548979198  24973891  881187550   881177866     False   
4766349     1548979198  47592150  881187546   881187304     False   
4766350     1548979198  57420342  881187549   881187524      True   
4766351     1548979199  59823847  881187551   881187083     False   

              user_text   user_id seconds_to_prev  curr_bytes delta_bytes  \
0                BD2412    196446         3696519        1195          -9   
1                



Bot revisions are those where `user_is_bot` is true. Question: How many bots made edits in January 2019? (Hint: The name of 
the user who made the revision is given in the user_text column. I am asking: how many unique 'user_text' values are there 
for rows where user_is_bot == True? There is a Pandas function that does this, but there are several reasonable approaches.)



In [6]:
df.user_is_bot.value_counts()

False    4241207
True      525145
Name: user_is_bot, dtype: int64

In [7]:
# number of unique bot ids

dfnew = df[df.user_is_bot]
len(dfnew.user_text.value_counts())

105

What % of bots edits are reverted and what % of human edits are reverted?  (Hint: This is already in the notebook.  Read the documentation for the crosstab command.  The column "is_reverted" tells you whether a revisions was reverted or not (according to a very particular discussion of "reverted", which we will discuss later).)


In [8]:
# percentage of bot edits reverted: 5%
# percentage of human edits reverted: 10%

pd.crosstab(df.user_is_bot, df.is_reverted, margins=True, normalize='index').round(decimals=3)

is_reverted,False,True
user_is_bot,,
False,0.898,0.102
True,0.950,0.050
All,0.903,0.097


What bot has the most revisions in the dataset? (Hint: already in the notebook. See: pd.DataFrame.value_counts.) What does Cydebot do?  Try to find the original bot approval discussion for Cydebot. (Hint: read about the bot approvals process, find the archived bot approvals list, look for Cydebot. Extra credit: Is Cydebot still running as of June 2021?  If not, what happened to it and why?)

In [9]:
dfnew.user_text.value_counts().head(1)

Cydebot    42742
Name: user_text, dtype: int64

Cydebot looks at discussions about category deletion, then automatically goes through and deletes mentions of those pages after discussions are concluded.

https://en.wikipedia.org/wiki/Special:Contributions/Cydebot
https://en.wikipedia.org/wiki/Wikipedia:Categories_for_discussion/Log/2020_January_2#Bunch_of_journal/magazine_people_categories

Cydebot was banned because it was deleting things even though no consensus was reached in the discussion (linked).

https://en.wikipedia.org/wiki/Wikipedia:Bots/Requests_for_approval/Cydebot_4
^^ approval discussion

Look at some revisions made by Cydebot.  Wikipedia let's you construct URLs with the rev_id directly (which is a poorly documented feature): use this to specifically look at changes made by Cydebot, e.g. https://en.wikipedia.org/w/index.php?diff=879519093  No, but really: what does Cydebot do?

In [15]:
dfcydebot = dfnew[dfnew.user_text == 'Cydebot'].sample(n=10)
print(dfcydebot.columns)

Index(['rev_timestamp', 'page_id', 'rev_id', 'prev_rev_id', 'is_minor',
       'user_text', 'user_id', 'seconds_to_prev', 'curr_bytes', 'delta_bytes',
       'has_edit_summary', 'is_reverted', 'is_revert',
       'is_reverted_to_by_other', 'is_self_reverted', 'is_self_revert',
       'revert_target_id', 'revert_set_size', 'revert_id', 'seconds_to_revert',
       'damaging_pred', 'goodfaith_pred', 'model_version', 'user_is_bot',
       'user_is_trusted', 'user_edit_count', 'page_rev_count',
       'page_namespace', 'is_page_redirect'],
      dtype='object')


In [21]:
#based on a couple of different revision IDs, we can see the Cydebot deletes a lot, and also makes revisions on user's "work-in-progress" pages?
#

dfcydebot.sample(n = 10)


,rev_timestamp,page_id,rev_id,prev_rev_id,is_minor,user_text,user_id,seconds_to_prev,curr_bytes,delta_bytes,...,seconds_to_revert,damaging_pred,goodfaith_pred,model_version,user_is_bot,user_is_trusted,user_edit_count,page_rev_count,page_namespace,is_page_redirect
4347699,1548746656,13784401,880751373,880750997,True,Cydebot,1215485,351,1471,-252,...,None,0.014709,0.989582,0.5.0,True,True,6519701,68254,2,1
1940765,1547416362,55718790,878246907,849436378,True,Cydebot,1215485,16319396,6810,-53,...,None,0.005553,0.995473,0.5.0,True,True,6495153,3,0,0
1974820,1547426610,32416024,878282849,439510229,True,Cydebot,1215485,236747927,2818,-53,...,None,0.006068,0.996073,0.5.0,True,True,6503542,1,2,0
2404731,1547657147,54298998,878731481,878331497,True,Cydebot,1215485,209384,1503,0,...,None,0.007849,0.994370,0.5.0,True,True,6512877,3,0,0
2628141,1547778259,13784401,878964230,878963648,True,Cydebot,1215485,358,3196,-40,...,None,0.010731,0.994886,0.5.0,True,True,6513675,68254,2,1
2012025,1547440641,20164147,878321543,876001366,True,Cydebot,1215485,1260784,3099,0,...,None,0.008863,0.992687,0.5.0,True,True,6511501,2,0,0
2833699,1547912615,26594019,879178159,836571200,True,Cydebot,1215485,24103905,3090,-36,...,None,0.008056,0.993464,0.5.0,True,True,6514673,3,0,0
1928936,1547412963,29541446,878234160,873678392,True,Cydebot,1215485,2621720,3545,-53,...,None,0.008099,0.993094,0.5.0,True,True,6492224,7,0,0
1960596,1547422531,44372820,878267805,633446107,True,Cydebot,1215485,131676599,562,-54,...,None,0.008430,0.994600,0.5.0,True,True,6500531,1,2,0
1990485,1547431700,34580519,878299155,474613789,True,Cydebot,1215485,219226077,3176,-53,...,None,0.004859,0.996749,0.5.0,True,True,6507908,1,2,0


In [ ]:
bdf[bdf.user_text =